In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
df = pd.read_parquet(r'C:\Users\PC\Documents\GitHub\Khoa-luan\train.parquet')
oos = pd.read_parquet(r'C:\Users\PC\Documents\GitHub\Khoa-luan\oos.parquet')
oot = pd.read_parquet(r'C:\Users\PC\Documents\GitHub\Khoa-luan\oot.parquet')

In [22]:
# Danh sách các biến cần ép kiểu về Category (Nominal & Binary)
categorical_cols = [
    'OCCUPATION_TYPE', 
    'PURCOD_MAX', 
    'PURCOD_MIN',
    'SOHUUNHA', 
    'NHANVIENBIDV',
    'FLAG_SALARY_ACC', 
    'FLAG_DEPOSIT', 
    'FLAG_CASH_ADVANCE',
    'HAS_SHORTTERM_LOAN', # Nếu có trong df final
    'HAS_LONGTERM_LOAN',  # Nếu có trong df final
    'BAD_CURRENT',        # Nếu giữ lại làm feature (thường là drop vì data leakage)
    'XULYNO'              # Nếu giữ lại
]

# Lọc những cột thực sự tồn tại trong df (đề phòng bạn đã drop bớt)
existing_cat_cols = [col for col in categorical_cols if col in df.columns]

# Chuyển đổi
for col in existing_cat_cols:
    df[col] = df[col].astype('str') # Chuyển về string để EBM hiểu là category
    oos[col] = oos[col].astype('str')
    oot[col] = oot[col].astype('str')

data leakage: bỏ các biến tương quan cao với target 

các obs đã xấu sẵn

In [27]:
# Danh sách các biến cần loại bỏ để khử đa cộng tuyến
cols_to_drop_final = [
    # 1. Deposit Group
    'N_AVG_DEPOSIT_6M', 'N_AVG_DD_12M', 'N_AVG_CD_12M',
    
    # 2. Balance & Limit Group
    'AFLIMT_MIN', 'AFLIMT_AVG', 
    'CBAL', 'CBAL_MAX', 'CBAL_MIN', 'CBALORG',
    
    # 3. Delinquency Group
    'MAX_DPD_12M', 'SUM_ALL_OD_12M', 'N_AVG_OVERDUE_CBAL_12M',
    
    # 4. Ratios Group
    'CBAL_TO_INC_12MON', 'PTI_RATIO', 'COLLATERAL_VALUE',
    
    # 5. Others
    'REMAINING_DURATION_MAX', 'TIME_TO_OP_MAX'
]

# Lọc chỉ những cột có trong X để tránh lỗi
cols_to_drop_final = [c for c in cols_to_drop_final if c in X.columns]

# Thực hiện Drop
X = X.drop(columns=cols_to_drop_final)
X_oos = X_oos.drop(columns=cols_to_drop_final)
X_oot = X_oot.drop(columns=cols_to_drop_final)

print(f"Đã bỏ {len(cols_to_drop_final)} biến.")
print(f"Số lượng biến còn lại: {X.shape[1]}")
print("Danh sách biến còn lại:", X.columns.tolist())

Đã bỏ 17 biến.
Số lượng biến còn lại: 37
Danh sách biến còn lại: ['C_GIOITINH', 'TRINHDO', 'TTHONNHAN', 'SOHUUNHA', 'NHANVIENBIDV', 'BASE_AUM', 'TUOI', 'INCOME', 'AFLIMT_MAX', 'CBAL_AVG', 'LTV', 'N_AVG_DEPOSIT_12M', 'FLAG_SALARY_ACC', 'FLAG_DEPOSIT', 'UTILIZATION_RATE', 'CNT_CREDIT_CARDS', 'AMT_CASH_ADVANCE_12M', 'FLAG_CASH_ADVANCE', 'PCT_PAYMENT_TO_BALANCE', 'CNT_MIN_PAY_6M', 'AVG_DAYS_PAST_DUE', 'DTI_RATIO', 'MOB', 'CNT_OTHER_PRODUCTS', 'LIMIT_TO_INCOME', 'AMT_VAR_6M', 'CBAL_SHORTTERM_LOAN', 'CBAL_LONGTERM_LOAN', 'CNT_DPD_30PLUS_6M', 'OCCUPATION_TYPE', 'DURATION_MAX', 'RATE_AVG', 'PURCOD_MAX', 'PURCOD_MIN', 'MAX_DPD_12M_OBS', 'AVG_OD_DPD_12M', 'REAL_GDP_GROWTH_12M']


In [28]:
drop_oos = [c for c in cols_to_drop_final if c in oos.columns]
drop_oot = [c for c in cols_to_drop_final if c in oot.columns]

# 2. Thực hiện bỏ cột
if drop_oos:
    oos = oos.drop(columns=drop_oos)
    print(f"   - Đã bỏ {len(drop_oos)} biến trong tập OOS.")

if drop_oot:
    oot = oot.drop(columns=drop_oot)
    print(f"   - Đã bỏ {len(drop_oot)} biến trong tập OOT.")

# 3. Kiểm tra lại số lượng cột
print(f"\nKiểm tra số lượng cột sau khi đồng bộ:")
print(f"   - Train (X): {X.shape[1]} biến")
print(f"   - OOS:       {oos.shape[1]} biến (bao gồm cả Target)")
print(f"   - OOT:       {oot.shape[1]} biến (bao gồm cả Target)")

   - Đã bỏ 17 biến trong tập OOS.
   - Đã bỏ 17 biến trong tập OOT.

Kiểm tra số lượng cột sau khi đồng bộ:
   - Train (X): 37 biến
   - OOS:       38 biến (bao gồm cả Target)
   - OOT:       38 biến (bao gồm cả Target)


outlier

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Chọn các biến số (Numeric) để kiểm tra
# Loại bỏ các biến đã chuyển thành object/category ở bước trước
numeric_cols = X.select_dtypes(include=[np.number]).columns

outlier_stats = []

for col in numeric_cols:
    # Bỏ qua các biến Binary (chỉ có 0 và 1)
    if X[col].nunique() <= 2:
        continue
        
    series = X[col].dropna()
    if len(series) == 0: continue
    
    # Tính các chỉ số
    p01 = np.percentile(series, 1)
    p25 = np.percentile(series, 25)
    median = np.percentile(series, 50)
    p75 = np.percentile(series, 75)
    p99 = np.percentile(series, 99)
    maximum = series.max()
    minimum = series.min()
    
    # Tính độ lệch (Skewness) giữa Max và P99
    # Nếu Max gấp nhiều lần P99 -> Khả năng cao là Outlier cực đoan
    ratio_max_p99 = maximum / p99 if p99 > 0 else 0
    
    outlier_stats.append({
        'Column': col,
        'Min': minimum,
        'P01': p01,
        'Median': median,
        'P99': p99,
        'Max': maximum,
        'Max/P99 Ratio': round(ratio_max_p99, 2)
    })

# 2. Tạo DataFrame báo cáo và sắp xếp theo độ lệch giảm dần
outlier_report = pd.DataFrame(outlier_stats)
outlier_report = outlier_report.sort_values(by='Max/P99 Ratio', ascending=False)

# 3. Hiển thị Top 15 biến có Outlier nặng nhất
print("=== BÁO CÁO PHÁT HIỆN OUTLIER (TOP 15) ===")
print(outlier_report.to_string(index=False))

=== BÁO CÁO PHÁT HIỆN OUTLIER (TOP 15) ===
                Column           Min           P01       Median          P99          Max  Max/P99 Ratio
     N_AVG_DEPOSIT_12M  5.129000e+03  1.067510e+05 1.622795e+06 3.214021e+07 6.877327e+08          21.40
              BASE_AUM  5.342900e+04  5.495779e+05 6.416078e+06 8.386845e+07 8.834219e+08          10.53
  AMT_CASH_ADVANCE_12M  0.000000e+00  0.000000e+00 0.000000e+00 7.053942e+08 4.399059e+09           6.24
     AVG_DAYS_PAST_DUE  0.000000e+00  0.000000e+00 0.000000e+00 1.900000e+01 1.170000e+02           6.16
   CBAL_SHORTTERM_LOAN  4.890000e+02  2.602907e+06 1.379224e+08 8.593129e+08 3.987872e+09           4.64
     CNT_DPD_30PLUS_6M  0.000000e+00  0.000000e+00 0.000000e+00 2.000000e+00 9.000000e+00           4.50
    CBAL_LONGTERM_LOAN  1.480000e+02  2.603787e+06 1.378922e+08 8.618660e+08 3.573867e+09           4.15
              CBAL_AVG  2.314456e+07  7.603128e+07 3.106165e+08 1.335149e+09 5.276239e+09           3.95
            

In [30]:
# 1. Hàm tính ngưỡng (Chỉ dùng dữ liệu Train)
def get_capping_rules(df, cols, percentile=0.995):
    rules = {}
    for col in cols:
        if col in df.columns:
            # Tính giá trị phân vị trên tập Train
            limit = df[col].quantile(percentile)
            rules[col] = limit
            print(f"   -> Rule {col:<25}: Cap @ {limit:,.0f}")
    return rules

# 2. Hàm áp dụng (Dùng cho Train, OOS, OOT)
def apply_capping(df, rules):
    df_clean = df.copy()
    for col, limit in rules.items():
        if col in df_clean.columns:
            # Chỉ thay thế giá trị > limit bằng limit (không đụng vào giá trị nhỏ hơn)
            df_clean[col] = df_clean[col].clip(upper=limit)
    return df_clean

# Danh sách biến cần xử lý
cols_to_cap = [
    'N_AVG_DEPOSIT_12M', 'BASE_AUM', 'AMT_CASH_ADVANCE_12M',
    'AVG_DAYS_PAST_DUE', 'CBAL_SHORTTERM_LOAN', 'CNT_DPD_30PLUS_6M',
    'CBAL_LONGTERM_LOAN', 'CBAL_AVG', 'AFLIMT_MAX'
]

print(">>> [1] Đang học ngưỡng Outlier từ tập TRAIN (X)...")
capping_rules = get_capping_rules(X, cols_to_cap, percentile=0.995)

print("\n>>> [2] Đang áp dụng ngưỡng vào Train, OOS, OOT...")
X_capped = apply_capping(X, capping_rules)
X_oos_capped = apply_capping(X_oos, capping_rules)
X_oot_capped = apply_capping(X_oot, capping_rules)

# Kiểm tra nhanh kết quả trên OOT
print(f"\nKiểm tra biến {cols_to_cap[0]} trên OOT:")
print(f"   - Max trước khi cap: {X_oot[cols_to_cap[0]].max():,.0f}")
print(f"   - Max sau khi cap:   {X_oot_capped[cols_to_cap[0]].max():,.0f} (Phải bằng ngưỡng Train)")

>>> [1] Đang học ngưỡng Outlier từ tập TRAIN (X)...
   -> Rule N_AVG_DEPOSIT_12M        : Cap @ 44,081,819
   -> Rule BASE_AUM                 : Cap @ 110,680,615
   -> Rule AMT_CASH_ADVANCE_12M     : Cap @ 880,361,297
   -> Rule AVG_DAYS_PAST_DUE        : Cap @ 24
   -> Rule CBAL_SHORTTERM_LOAN      : Cap @ 1,032,608,513
   -> Rule CNT_DPD_30PLUS_6M        : Cap @ 3
   -> Rule CBAL_LONGTERM_LOAN       : Cap @ 1,035,228,584
   -> Rule CBAL_AVG                 : Cap @ 1,572,500,396
   -> Rule AFLIMT_MAX               : Cap @ 2,349,415,899

>>> [2] Đang áp dụng ngưỡng vào Train, OOS, OOT...

Kiểm tra biến N_AVG_DEPOSIT_12M trên OOT:
   - Max trước khi cap: 692,326,582
   - Max sau khi cap:   44,081,819 (Phải bằng ngưỡng Train)


In [31]:
X_capped.shape

(1137807, 37)

In [32]:
X_oos_capped.shape

(300317, 37)

In [33]:
X_oot_capped.shape

(302113, 37)

In [34]:
y_train = df['BAD_NEXT_12M']
y_oos = oos['BAD_NEXT_12M']
y_oot = oot['BAD_NEXT_12M']

X_capped.to_parquet('X_train.parquet', index=False)
X_oos_capped.to_parquet('X_oos.parquet', index=False)
X_oot_capped.to_parquet('X_oot.parquet', index=False)
y_train.to_frame().to_parquet('y_train.parquet', index=False)
y_oos.to_frame().to_parquet('y_oos.parquet', index=False)
y_oot.to_frame().to_parquet('y_oot.parquet', index=False)